<a href="https://colab.research.google.com/github/perezsam/DM19-Lab2-Homework/blob/master/twitter_sentiment_prediction_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install emoji

     |████████████████████████████████| 51kB 1.7MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42175 sha256=52cea8f65d5c3d9a6cf8a782bf878bbb64a2887ab34041d647a43fd9a49ac493
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [0]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import json
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Dropout

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop

from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

pd.set_option('display.max_colwidth', -1)
from time import time
import re
import string
import os
import emoji
from pprint import pprint
import collections

sns.set(style="darkgrid")
sns.set(font_scale=1.3)

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

import gensim

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

np.random.seed(37)
KAGGLE_ENV = os.getcwd == '/kaggle/working'

Using TensorFlow backend.


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
import json
from pandas.io.json import json_normalize

##Loading raw dataset from kaggle##

In [0]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

In [0]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 64 Nov 27 21:07 kaggle.json


In [0]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
# List available datasets.
#!kaggle datasets list

In [0]:
# Copy the private data set locally.
!kaggle datasets download -d cssamuel/dmlab2

 90% 134M/149M [00:00<00:00, 178MB/s]
100% 149M/149M [00:00<00:00, 192MB/s]


In [0]:
ls

data/  dmlab2.zip  kaggle.json  output/  sample_data/


In [0]:
!mv dmlab2.zip ./data/

In [0]:
pwd

'/content'

In [0]:
cd data

/content/data


In [0]:
!unzip dmlab2.zip

Archive:  dmlab2.zip
  inflating: data_identification.csv  
  inflating: emotion.csv             
  inflating: sampleSubmission.csv    
  inflating: tweets_DM.json          


In [0]:
pwd

'/content/data'

In [0]:
cd ..

/content


In [0]:
ls

data/  kaggle.json  output/  sample_data/


## Data preprocessing ##

In [0]:
# read data files
raw_data=pd.read_json("./data/tweets_DM.json",lines=True)
tweets=json_normalize(data=raw_data['_source'])
identify=pd.read_csv("./data/data_identification.csv")
emotion=pd.read_csv("./data/emotion.csv")

In [0]:
pwd

'/content'

In [0]:
# rename column names
tweets=tweets.rename(index=str,columns={"tweet.text":"text", "tweet.tweet_id":"tweet_id",
                                       "tweet.hashtags":"hashtags"})
# add identify tags to dataframe
tweets=pd.merge(tweets,identify, on="tweet_id")

#get training set and test set
train_df=tweets[tweets["identification"] == "train"]
test_df=tweets[tweets["identification"] == "test"]

#add emotion column
train_df=pd.merge(train_df,emotion, on="tweet_id")
#test_df["emotion"]=""

#drop identification tags
train_df.drop(columns=["identification"],inplace=True)
test_df.drop(columns=["identification"],inplace=True)

#use tweet_id as index
train_df.set_index("tweet_id",inplace=True)
test_df.set_index("tweet_id",inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Save training and testing dataset to a pickle file

In [0]:
# save to pickle file
train_df.to_pickle("./output/train_df.pkl")
test_df.to_pickle("./output/test_df.pkl")

Load data training data for our Neural Network model, selecting only useful data (i.e. text and emotion)

In [0]:
## load a pickle file
df = pd.read_pickle("./output/train_df.pkl")
df = df.reindex(np.random.permutation(df.index))  
df = df[['text', 'emotion']]

## select a sample of 250,000 due time and memory constraint in google colab
df = df.sample(250000)

In [0]:
## same class used in simple text minig to clean training and testing data set
import nltk
nltk.download('stopwords')

class CleanText(BaseEstimator, TransformerMixin):
   
    def remove_mentions(self, input_text):
        return re.sub(r'@\w+', '', input_text)
    
    def remove_urls(self, input_text):
        return re.sub(r'http.?://[^\s]+[\s]?', '', input_text)
    
    def emoji_oneword(self, input_text):
        # By compressing the underscore, the emoji is kept as one word
        return input_text.replace('_','')
    
    def remove_punctuation(self, input_text):
        # Make translation table
        punct = string.punctuation
        trantab = str.maketrans(punct, len(punct)*' ')  # Every punctuation symbol will be replaced by a space
        return input_text.translate(trantab)

    def remove_digits(self, input_text):
        return re.sub('\d+', '', input_text)
    
    def to_lower(self, input_text):
        return input_text.lower()
    
    def remove_stopwords(self, input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words) 
    
    def stemming(self, input_text):
        porter = PorterStemmer()
        words = input_text.split() 
        stemmed_words = [porter.stem(word) for word in words]
        return " ".join(stemmed_words)
    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, **transform_params):
        clean_X = X.apply(self.remove_mentions).apply(self.remove_urls).apply(self.emoji_oneword).apply(self.remove_punctuation).apply(self.remove_digits).apply(self.to_lower).apply(self.remove_stopwords).apply(self.stemming)
        return clean_X

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Text cleaning for training dataset ###

In [0]:
ct = CleanText()
sr_clean = ct.fit_transform(df.text)
sr_clean.sample(5)

tweet_id
0x341116    hmm mayb lh                                                                
0x227e4b    like eat organ clean make bodi feel free less bodi tension build also go lh
0x21483d    miss stream work kill lh                                                   
0x36a7f8    look forward explor new look later today lh plush                          
0x21b2a5    believ lh lh                                                               
Name: text, dtype: object

In [0]:
empty_clean = sr_clean == ''
print('{} records have no words left after text cleaning'.format(sr_clean[empty_clean].count()))
sr_clean.loc[empty_clean] = '[no_text]'

0 records have no words left after text cleaning


In [0]:
df_model = df
df_model['clean_text'] = sr_clean
df_model.columns.tolist()

['text', 'emotion', 'clean_text']

In [0]:
clean_text_train = df_model[['clean_text', 'emotion']]

In [0]:
clean_text_train.to_csv('./data/train.csv')

In [0]:
pd.read_csv('./data/train.csv')

,tweet_id,clean_text,emotion
0,0x2cad92,shoot vega sad pray lh la vega lh shoot vega violenc,sadness
1,0x372564,uni skip class studi midterm lh,anticipation
2,0x1f3a0a,close sell lot eurusd pip total today pip lh lh lh lh lh,joy
3,0x2508e3,bore realli feel like move bore lh ohioproblem,disgust
4,0x242436,miss much lh wanna one 흥얼흥얼,joy
...,...,...,...
249995,0x26525d,slot left ☺️ dm order twice twicetagram aroha astro lh init carpediem epikhigh wevedonesomethingwond,anticipation
249996,0x2850b6,never let ladi sleep lh sad,sadness
249997,0x32f577,launch day today dplaunch lh winerecept prioriti,anticipation
249998,0x293c3b,must one mani brat lh titan,surprise


##Text cleaning for testing dataset##

In [0]:
## load a pickle file
model_test_df = pd.read_pickle("./output/test_df.pkl")
#df = df.reindex(np.random.permutation(df.index))  
model_test_df = model_test_df[['text', 'emotion']]
#model_test_df = model_test_df.sample(50000)

In [0]:
sr_clean_test = ct.fit_transform(model_test_df.text)
sr_clean_test.sample(5)

tweet_id
0x2c18a7    alreadi saw least fan anoth group claim comeback trailer better comeback mv 😄😊 lh 
0x355c58    hommi daxton download game onto xbox via wifi thank use fast wifi month lh        
0x332513    know ur peopl stop react gonegirl overit lh                                       
0x20db8b    trump crazi seri tweet puerto rico dedic golf trophi peopl realli itmfa surreal lh
0x31abec    god keep bless first new place start work tomorrow 🙏🏽❣️ lh lh                     
Name: text, dtype: object

In [0]:
empty_clean_t = sr_clean_test == ''
print('{} records have no words left after text cleaning'.format(sr_clean_test[empty_clean_t].count()))
sr_clean_test.loc[empty_clean_t] = '[no_text]'

0 records have no words left after text cleaning


In [0]:
df_model_test = model_test_df
df_model_test['clean_text'] = sr_clean_test
df_model_test.columns.tolist()

['text', 'emotion', 'clean_text']

In [0]:
clean_text_test = df_model_test[['clean_text']]

In [0]:
clean_text_test.to_csv('./data/test.csv')

##Train Neural Network##

In [0]:
def convert_text_to_index_array(text):
    return [dictionary[word] for word in kpt.text_to_word_sequence(text)]

def data_prepare(training_file_path):

    dataset = pd.read_csv(training_file_path)
    reviews = []
    labels = []    
    
    # Enconding Categorical Data     
    labelencoder_y = LabelEncoder()
    dataset['emotion'] = labelencoder_y.fit_transform(dataset['emotion'])
    cLen = len(dataset['clean_text'])
        
    for i in range(0,cLen):
        review = dataset['clean_text'][i]
        reviews.append(review) 
        label = dataset["emotion"][i]
        labels.append(label)    
    labels = np.asarray(labels)
    return reviews,labels


train_file_path = "./data/train.csv"
[reviews,labels] = data_prepare(train_file_path)

In [0]:
# Create Dictionary of words and their indices
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(reviews)
dictionary = tokenizer.word_index

# save dictionary
with open('dictionary.json','w') as dictionary_file:
    json.dump(dictionary,dictionary_file)

# Replace words of each text review to indices
allWordIndices = []
for num,text in enumerate(reviews):
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

In [0]:
#OUT OF MEMORY
# Convert the index sequences into binary bag of words vector (one hot encoding) 
allWordIndices = np.asarray(allWordIndices)
train_X = tokenizer.sequences_to_matrix(allWordIndices, mode='binary')
labels = keras.utils.to_categorical(labels,num_classes=8)

In [0]:
# Creating Dense Neural Networl Model
model = Sequential()
model.add(Dense(256, input_shape=(max_words,), activation='elu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='elu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

print (model.summary())

model.compile(loss='categorical_crossentropy',
  optimizer='sgd',
  metrics=['accuracy'])





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               2560256   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 1032      
Total params: 2,594,184
Trainable params: 2,594,184
Non-trainable params: 0
_________________________________________________________

In [0]:
# Training the Model
model.fit(train_X, labels,
  batch_size=32,
  epochs=10,
  verbose=1,
  validation_split=0.1,
  shuffle=True)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 225000 samples, validate on 25000 samples
Epoch 1/10





225000/225000 [==============================] - 42s 187us/step - loss: 1.6556 - acc: 0.4005 - val_loss: 1.5337 - val_acc: 0.4364
Epoch 2/10
225000/225000 [==============================] - 37s 163us/step - loss: 1.5060 - acc: 0.4540 - val_loss: 1.4568 - val_acc: 0.4705
Epoch 3/10
225000/225000 [==============================] - 37s 162us/step - loss: 1.4497 - acc: 0.4763 - val_loss: 1.4201 - val_acc: 0.4840
Epoch 4/10
225000/225000 [==============================] - 36s 162us/step - loss: 1.4167 - acc: 0.4882 - val_loss: 1.4045 - val_acc: 0.4889
Epoch 5/10
225000/225000 [==============================] - 36s 162us/step - loss: 1.3956 - acc: 0.4965 - val_loss: 1.3928 - val_acc: 0.4983
Epoch 6/10
225000/225000 [==============================] - 36s 162us/step - loss: 1.3795 - acc: 0.5035 - val_loss: 1.3818 - val_acc: 0.5022
E

In [0]:
# Save model to disk
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')   

##Predict on testing data##

In [0]:
from keras.models import model_from_json

In [0]:
def convert_text_to_index_array(text):
    words = kpt.text_to_word_sequence(text)
    wordIndices = []
    for word in words:
        if word in dictionary:
            wordIndices.append(dictionary[word])
    return wordIndices

In [0]:
# Load the dictionary
labels = ["anger", "anticipation", "disgust",	"fear",	"joy", "sadness", "surprise", "trust"]
with open('dictionary.json', 'r') as dictionary_file:
    dictionary = json.load(dictionary_file)

In [0]:
# Load trained model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights('model.h5')

testset = pd.read_csv("./data/test.csv")    
cLen = len(testset['clean_text'])
tokenizer = Tokenizer(num_words=10000)

In [0]:
# Predict happiness for each review in test.csv
y_pred = []   
for i in range(0,cLen):
    review = testset['clean_text'][i]
    testArr = convert_text_to_index_array(review)   
    input = tokenizer.sequences_to_matrix([testArr], mode='binary')
    pred = model.predict(input)
    #print pred[0][np.argmax(pred)] * 100, labels[np.argmax(pred)]
    y_pred.append(labels[np.argmax(pred)])

In [0]:
# Write the results in submission csv file
raw_data = {'id': testset['tweet_id'], 
        'emotion': y_pred}
df = pd.DataFrame(raw_data, columns = ['id', 'emotion'])
df.to_csv('submission_model1.csv', sep=',',index=False)